# Demand Generation

In [1]:
# imports
import pandas as pd
import numpy as np

In [2]:
# variables
large_grid_models_input_filepath = './pickles/large_grid_ml_models.pickle'
small_grid_models_input_filepath = './pickles/small_grid_ml_models.pickle'
voronoi_grid_models_input_filepath = './pickles/voronoi_grid_ml_models.pickle'

data_input_filepath = './pickles/reservations_testing.pickle'
training_data_input_filepath =  './pickles/reservations_training.pickle'

large_grid_poisson_demand_output_filepath = './pickles/demand/large_grid_poisson_demand_prediction.pickle'
large_grid_negative_binomial_demand_output_filepath = './pickles/demand/large_grid_negative_binomial_demand_prediction.pickle'
small_grid_poisson_demand_output_filepath = './pickles/demand/small_grid_poisson_demand_prediction.pickle'
small_grid_negative_binomial_demand_output_filepath = './pickles/demand/small_grid_negative_binomial_demand_prediction.pickle'
voronoi_grid_poisson_demand_output_filepath = './pickles/demand/voronoi_grid_poisson_demand_prediction.pickle'
voronoi_grid_negative_binomial_demand_output_filepath = './pickles/demand/voronoi_grid_negative_binomial_demand_prediction.pickle'

In [3]:
# load data
small_grid_models = pd.read_pickle(small_grid_models_input_filepath)
large_grid_models = pd.read_pickle(large_grid_models_input_filepath)
voronoi_grid_models = pd.read_pickle(voronoi_grid_models_input_filepath)

In [4]:
# load reservation data
input_data = pd.read_pickle(data_input_filepath)
input_data['startTime'] = pd.to_datetime(input_data['startTime'])
input_data.set_index('startTime', inplace=True)
input_data.drop(columns=['endTime', 'temperature', 'precipitation', 'startLat', 'startLon', 'endLat', 'endLon'], inplace=True)
input_data.head()

,voronoi_grid_id,small_grid_id,large_grid_id,community_small_grid_id,community_voronoi_grid_id
startTime,,,,,
2018-04-30 10:43:49,448,5059,195,13,0
2022-09-16 15:46:43,339,3271,127,7,0
2019-09-07 20:00:38,949,6435,267,8,7
2022-06-24 11:28:29,454,4972,195,7,7
2018-07-16 15:15:15,155,4295,179,7,4


In [5]:
input_training_data = pd.read_pickle(training_data_input_filepath)
input_training_data['startTime'] = pd.to_datetime(input_training_data['startTime'])
input_training_data.set_index('startTime', inplace=True)
input_training_data.drop(columns=['endTime', 'temperature', 'precipitation', 'startLat', 'startLon', 'endLat', 'endLon'], inplace=True)
input_training_data.head()

,voronoi_grid_id,small_grid_id,large_grid_id,community_small_grid_id,community_voronoi_grid_id
startTime,,,,,
2023-02-27 14:05:52,263,4717,195,7,0
2021-05-28 13:30:25,863,6600,266,0,4
2022-08-27 17:47:21,468,5484,212,0,7
2021-07-22 16:31:23,287,3963,163,6,9
2018-09-15 16:14:54,335,3269,126,12,4


# Large Grid ML Models

In [6]:
# transform data
large_grid_demand_data = input_data.resample('3H').large_grid_id.value_counts().unstack().fillna(0)
large_grid_demand_data['hour'] = large_grid_demand_data.index.hour
large_grid_demand_data['day_of_week'] = large_grid_demand_data.index.dayofweek
large_grid_demand_data['month'] = large_grid_demand_data.index.month
large_grid_demand_data.reset_index(inplace=True, drop=False)
large_grid_demand_data = large_grid_demand_data[
    (large_grid_demand_data['month'] != 1) &
    (large_grid_demand_data['month'] != 2) &
    (large_grid_demand_data['month'] != 11) &
    (large_grid_demand_data['month'] != 12)
]
large_grid_demand_data.head()

large_grid_id,startTime,12,21,53,69,70,76,90,103,107,...,284,297,298,299,300,301,313,hour,day_of_week,month
0,2017-10-14 12:00:00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,5,10
1,2017-10-14 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15,5,10
2,2017-10-14 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,5,10
3,2017-10-14 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,5,10
4,2017-10-15 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,6,10


In [7]:
large_grid_poisson_predicted_demand = pd.DataFrame()
large_grid_negative_binomial_predicted_demand = pd.DataFrame()

large_grid_poisson_predicted_demand['timestamp'] = large_grid_demand_data.startTime
large_grid_negative_binomial_predicted_demand['timestamp'] = large_grid_demand_data.startTime

x = large_grid_demand_data[['hour', 'day_of_week', 'month']]
x = pd.get_dummies(x, columns=['hour', 'day_of_week', 'month'], drop_first=True)
for i in range(len(large_grid_models.index)):
    poisson_prediction = large_grid_models.poisson_model[i].predict(x)
    negative_binomial_prediction = large_grid_models.negative_binomial_model[i].predict(x)

    large_grid_poisson_predicted_demand[large_grid_models.cell[i]] = poisson_prediction
    large_grid_negative_binomial_predicted_demand[large_grid_models.cell[i]] = negative_binomial_prediction
large_grid_poisson_predicted_demand.head()

,timestamp,195,266,212,163,126,111,127,109,161,...,199,137,234,124,333,299,132,69,182,12
0,2017-10-14 12:00:00,0.467590,0.120662,0.233912,0.148309,0.877903,0.011163,0.567526,0.169160,1.087699,...,1.203686e-04,3.222315e-06,5.551061e-06,1.235775e-10,1.247627e-06,5.315257e-09,9.895447e-08,6.400777e-07,6.029670e-08,2.279266e-10
1,2017-10-14 15:00:00,0.595860,0.182780,0.367664,0.241795,1.281619,0.018331,0.840228,0.250466,1.196791,...,2.369359e-04,1.474516e-04,2.748215e-06,3.437231e-08,6.899338e-08,5.095876e-09,5.665706e-07,2.243172e-06,3.169538e-06,2.382457e-08
2,2017-10-14 18:00:00,0.366629,0.096230,0.204596,0.131406,0.721692,0.012694,0.481221,0.127169,0.661329,...,2.451119e-04,1.683356e-04,2.730139e-06,1.192115e-08,7.089240e-08,1.063116e-08,1.933066e-07,1.683959e-06,6.087815e-08,2.279218e-10
3,2017-10-14 21:00:00,0.108745,0.030246,0.057757,0.040290,0.246705,0.002134,0.130581,0.030229,0.307402,...,1.147403e-06,3.834831e-06,3.651018e-06,1.743936e-10,8.108581e-08,3.340637e-11,9.138022e-10,2.011115e-09,7.829689e-08,2.841300e-10
4,2017-10-15 06:00:00,0.153784,0.055332,0.153811,0.051593,0.570707,0.014458,0.314064,0.099979,0.140491,...,9.388826e-07,5.274224e-08,5.854028e-07,1.292106e-10,6.870069e-07,2.491744e-11,8.233381e-07,1.310713e-13,3.646617e-06,2.357985e-10


In [8]:
large_grid_negative_binomial_predicted_demand.head()

,timestamp,195,266,212,163,126,111,127,109,161,...,199,137,234,124,333,299,132,69,182,12
0,2017-10-14 12:00:00,0.470319,0.124824,0.236775,0.153238,0.882520,0.011558,0.566619,0.173775,1.121703,...,1.248117e-04,3.109211e-06,1.490712e-06,8.987208e-11,3.965684e-08,3.336250e-09,1.542198e-07,9.662232e-07,2.436497e-08,9.932665e-11
1,2017-10-14 15:00:00,0.590918,0.191807,0.376260,0.252234,1.284710,0.019147,0.842325,0.259727,1.220102,...,2.481769e-04,1.545860e-04,7.303251e-07,2.992540e-08,3.916270e-10,3.175772e-09,9.107948e-07,3.841076e-06,2.472953e-06,1.288846e-08
2,2017-10-14 18:00:00,0.358560,0.098193,0.205427,0.135080,0.717411,0.013214,0.475091,0.128626,0.667204,...,2.575143e-04,1.777511e-04,7.500086e-07,9.899008e-09,4.085622e-10,6.572042e-09,3.003830e-07,2.875592e-06,2.455065e-08,9.941850e-11
3,2017-10-14 21:00:00,0.105760,0.030339,0.057092,0.040239,0.245163,0.002193,0.127470,0.030254,0.308557,...,5.083091e-07,3.680852e-06,9.381999e-07,1.275201e-10,5.276511e-10,2.006909e-11,1.555312e-09,7.050257e-09,3.217887e-08,1.259928e-10
4,2017-10-15 06:00:00,0.150171,0.055819,0.155782,0.051063,0.561631,0.015119,0.312718,0.100976,0.137146,...,4.030243e-07,4.641748e-08,6.885325e-09,9.369906e-11,4.939375e-08,1.520271e-11,1.371525e-06,2.073253e-12,2.799887e-06,1.037104e-10


In [9]:
large_grid_poisson_predicted_demand.to_pickle(large_grid_poisson_demand_output_filepath)
large_grid_negative_binomial_predicted_demand.to_pickle(large_grid_negative_binomial_demand_output_filepath)

# Small Grid ML Models

In [10]:
# transform data
small_grid_demand_data = input_data.resample('3H').small_grid_id.value_counts().unstack().fillna(0)
small_grid_demand_data['hour'] = small_grid_demand_data.index.hour
small_grid_demand_data['day_of_week'] = small_grid_demand_data.index.dayofweek
small_grid_demand_data['month'] = small_grid_demand_data.index.month
small_grid_demand_data.reset_index(inplace=True, drop=False)
small_grid_demand_data.sort_index(inplace=True)
small_grid_demand_data = small_grid_demand_data[
    (small_grid_demand_data['month'] != 1) &
    (small_grid_demand_data['month'] != 2) &
    (small_grid_demand_data['month'] != 11) &
    (small_grid_demand_data['month'] != 12)
]
small_grid_demand_data.head()

small_grid_id,startTime,61,615,616,700,701,785,788,789,1626,...,7543,7544,7606,7607,7626,7627,7770,hour,day_of_week,month
0,2017-10-14 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,5,10
1,2017-10-14 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15,5,10
2,2017-10-14 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,5,10
3,2017-10-14 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,5,10
4,2017-10-15 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,6,10


In [11]:
small_grid_models.head()

,community,poisson_model,negative_binomial_model
0,0,<statsmodels.genmod.generalized_linear_model.G...,<statsmodels.genmod.generalized_linear_model.G...
1,1,<statsmodels.genmod.generalized_linear_model.G...,<statsmodels.genmod.generalized_linear_model.G...
2,2,<statsmodels.genmod.generalized_linear_model.G...,<statsmodels.genmod.generalized_linear_model.G...
3,3,<statsmodels.genmod.generalized_linear_model.G...,<statsmodels.genmod.generalized_linear_model.G...
4,4,<statsmodels.genmod.generalized_linear_model.G...,<statsmodels.genmod.generalized_linear_model.G...


In [12]:
small_grid_community_dict = input_training_data.groupby('community_small_grid_id')['small_grid_id'].apply(lambda x: list(set(x))).to_dict()
small_grid_community_dict

{0: [4111,
  4112,
  3601,
  3602,
  3091,
  3603,
  5653,
  3606,
  4116,
  5656,
  3097,
  3098,
  5146,
  4124,
  3613,
  3099,
  4127,
  5147,
  4640,
  3101,
  5401,
  5912,
  4198,
  5227,
  5739,
  3181,
  5742,
  3182,
  3692,
  3184,
  3701,
  4283,
  3261,
  3777,
  5315,
  3268,
  6343,
  3272,
  3273,
  3787,
  4299,
  3604,
  3608,
  3005,
  4369,
  3346,
  3347,
  5397,
  5398,
  4887,
  5911,
  5400,
  3863,
  2842,
  5403,
  3356,
  5402,
  2843,
  4896,
  2844,
  3871,
  1827,
  2845,
  3357,
  5655,
  6439,
  3431,
  4456,
  5481,
  5480,
  5995,
  3433,
  2920,
  5486,
  5483,
  5484,
  3948,
  3439,
  5482,
  3953,
  4974,
  4976,
  3958,
  2931,
  3445,
  2930,
  2934,
  2932,
  3434,
  2926,
  2927,
  2929,
  3441,
  3516,
  3517,
  5566,
  5567,
  3520,
  3518,
  4542,
  6081,
  3012,
  5061,
  3014,
  3015,
  6600,
  3011,
  3530,
  6599,
  3532,
  3354,
  3864],
 1: [7105, 6593],
 2: [4642, 6179, 5157, 5670, 6184, 3848, 4042, 7371, 7116, 5931, 6095, 2846],
 3: 

In [13]:
small_grid_models.poisson_model[0].model.exog_names

['hour_3',
 'hour_6',
 'hour_9',
 'hour_12',
 'hour_15',
 'hour_18',
 'hour_21',
 'day_of_week_1',
 'day_of_week_2',
 'day_of_week_3',
 'day_of_week_4',
 'day_of_week_5',
 'day_of_week_6',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'month_10',
 'grid_cell_2842',
 'grid_cell_2843',
 'grid_cell_2844',
 'grid_cell_2845',
 'grid_cell_2920',
 'grid_cell_2926',
 'grid_cell_2927',
 'grid_cell_2929',
 'grid_cell_2930',
 'grid_cell_2931',
 'grid_cell_2932',
 'grid_cell_2934',
 'grid_cell_3005',
 'grid_cell_3011',
 'grid_cell_3012',
 'grid_cell_3014',
 'grid_cell_3015',
 'grid_cell_3091',
 'grid_cell_3097',
 'grid_cell_3098',
 'grid_cell_3099',
 'grid_cell_3101',
 'grid_cell_3181',
 'grid_cell_3182',
 'grid_cell_3184',
 'grid_cell_3261',
 'grid_cell_3268',
 'grid_cell_3272',
 'grid_cell_3273',
 'grid_cell_3346',
 'grid_cell_3347',
 'grid_cell_3354',
 'grid_cell_3356',
 'grid_cell_3357',
 'grid_cell_3431',
 'grid_cell_3433',
 'grid_cell_3434',
 'grid_cell_3439',
 'gr

In [14]:
import warnings
from pandas.errors import PerformanceWarning
warnings.simplefilter(action='ignore', category=PerformanceWarning)

small_grid_poisson_predicted_demand = pd.DataFrame()
small_grid_negative_binomial_predicted_demand = pd.DataFrame()

small_grid_poisson_predicted_demand['timestamp'] = small_grid_demand_data.startTime
small_grid_negative_binomial_predicted_demand['timestamp'] = small_grid_demand_data.startTime

for (community, cells) in small_grid_community_dict.items():
    print(f'Training model for community {community}')

    x = small_grid_demand_data[['hour', 'day_of_week', 'month']]
    x = pd.get_dummies(x, columns=['hour', 'day_of_week', 'month'], drop_first=True)

    exog = x.copy()
    columns = small_grid_models.poisson_model[community].model.exog_names

    # Collect all columns to be added in a list
    columns_to_add = []

    for col in columns:
        if col not in exog.columns:
            columns_to_add.append(pd.Series(False, name=col, index=exog.index))

    # Concatenate all columns at once
    if columns_to_add:
        exog = pd.concat([exog] + columns_to_add, axis=1)

    for cell in cells:
        if 'grid_cell_' + str(cell) in exog.columns:
            exog['grid_cell_' + str(cell)] = True
        poisson_prediction = small_grid_models.poisson_model[community].predict(exog)
        negative_binomial_prediction = small_grid_models.negative_binomial_model[community].predict(exog)

        small_grid_poisson_predicted_demand[cell] = poisson_prediction
        small_grid_negative_binomial_predicted_demand[cell] = negative_binomial_prediction

small_grid_poisson_predicted_demand.head()

Training model for community 0
Training model for community 1
Training model for community 2
Training model for community 3
Training model for community 4
Training model for community 5
Training model for community 6
Training model for community 7
Training model for community 8
Training model for community 9
Training model for community 10
Training model for community 11
Training model for community 12
Training model for community 13
Training model for community 14
Training model for community 15
Training model for community 16
Training model for community 17
Training model for community 18
Training model for community 19


,timestamp,4111,4112,3601,3602,3091,3603,5653,3606,4116,...,2919,5743,4721,6609,4819,6100,5843,785,6519,5823
0,2017-10-14 12:00:00,0.010809,0.000645,0.000041,3.114549e-06,2.385469e-07,2.347137e-08,1.943340e-09,2.517181e-10,1.921091e-11,...,2.719869e-07,1.049621e-08,6.029599e-10,1.425258e-12,4.319903e-14,3.655456e-15,2.149310e-16,2.149310e-16,1.675588e-17,4.407368e-18
1,2017-10-14 15:00:00,0.015023,0.000897,0.000057,4.328938e-06,3.315584e-07,3.262306e-08,2.701065e-09,3.498651e-10,2.670142e-11,...,3.006377e-07,1.160187e-08,6.664750e-10,1.575393e-12,4.774957e-14,4.040518e-15,2.375716e-16,2.375716e-16,1.852093e-17,4.871636e-18
2,2017-10-14 18:00:00,0.007857,0.000469,0.000030,2.264032e-06,1.734049e-07,1.706184e-08,1.412655e-09,1.829792e-10,1.396482e-11,...,2.711342e-07,1.046330e-08,6.010695e-10,1.420789e-12,4.306359e-14,3.643996e-15,2.142572e-16,2.142572e-16,1.670335e-17,4.393550e-18
3,2017-10-14 21:00:00,0.002712,0.000162,0.000010,7.814183e-07,5.984974e-08,5.888802e-09,4.875702e-10,6.315429e-11,4.819883e-12,...,1.549225e-07,5.978595e-09,3.434432e-10,8.118202e-13,2.460597e-14,2.082131e-15,1.224237e-16,1.224237e-16,9.544075e-18,2.510417e-18
4,2017-10-15 06:00:00,0.006723,0.000401,0.000026,1.937082e-06,1.483633e-07,1.459793e-08,1.208653e-09,1.565551e-10,1.194815e-11,...,4.920249e-08,1.898767e-09,1.090756e-10,2.578294e-13,7.814713e-15,6.612727e-16,3.888106e-17,3.888106e-17,3.031143e-18,7.972937e-19


In [15]:
small_grid_negative_binomial_predicted_demand.head()

,timestamp,4111,4112,3601,3602,3091,3603,5653,3606,4116,...,2919,5743,4721,6609,4819,6100,5843,785,6519,5823
0,2017-10-14 12:00:00,0.011091,0.000552,0.000029,1.849563e-06,1.181095e-07,9.679223e-09,6.686495e-10,7.204619e-11,4.583934e-12,...,3.209294e-07,1.885331e-08,1.140667e-09,3.403674e-12,9.673805e-14,9.077714e-15,5.484880e-16,5.484880e-16,5.336810e-17,1.495167e-17
1,2017-10-14 15:00:00,0.015587,0.000776,0.000041,2.599341e-06,1.659888e-07,1.360300e-08,9.397073e-10,1.012523e-10,6.442174e-12,...,3.651228e-07,2.144949e-08,1.297742e-09,3.872375e-12,1.100593e-13,1.032775e-14,6.240171e-16,6.240171e-16,6.071712e-17,1.701058e-17
2,2017-10-14 18:00:00,0.008013,0.000399,0.000021,1.336307e-06,8.533396e-08,6.993227e-09,4.830985e-10,5.205328e-11,3.311887e-12,...,3.211251e-07,1.886480e-08,1.141362e-09,3.405749e-12,9.679703e-14,9.083248e-15,5.488224e-16,5.488224e-16,5.340064e-17,1.496078e-17
3,2017-10-14 21:00:00,0.002710,0.000135,0.000007,4.519785e-07,2.886246e-08,2.365316e-09,1.633982e-10,1.760596e-11,1.120178e-12,...,2.035799e-07,1.195950e-08,7.235762e-10,2.159103e-12,6.136528e-14,5.758401e-15,3.479305e-16,3.479305e-16,3.385378e-17,9.484514e-18
4,2017-10-15 06:00:00,0.006835,0.000340,0.000018,1.139893e-06,7.279137e-08,5.965346e-09,4.120915e-10,4.440237e-11,2.825098e-12,...,4.607237e-08,2.706566e-09,1.637532e-10,4.886287e-13,1.388764e-14,1.303189e-15,7.874049e-17,7.874049e-17,7.661481e-18,2.146449e-18


In [16]:
small_grid_poisson_predicted_demand.to_pickle(small_grid_poisson_demand_output_filepath)
small_grid_negative_binomial_predicted_demand.to_pickle(small_grid_negative_binomial_demand_output_filepath)

# Voronoi Grid ML Models

In [17]:
# transform data
voronoi_grid_demand_data = input_data.resample('3H').voronoi_grid_id.value_counts().unstack().fillna(0)
voronoi_grid_demand_data['hour'] = voronoi_grid_demand_data.index.hour
voronoi_grid_demand_data['day_of_week'] = voronoi_grid_demand_data.index.dayofweek
voronoi_grid_demand_data['month'] = voronoi_grid_demand_data.index.month
voronoi_grid_demand_data.reset_index(inplace=True, drop=False)
voronoi_grid_demand_data.sort_index(inplace=True)
voronoi_grid_demand_data = voronoi_grid_demand_data[
    (voronoi_grid_demand_data['month'] != 1) &
    (voronoi_grid_demand_data['month'] != 2) &
    (voronoi_grid_demand_data['month'] != 11) &
    (voronoi_grid_demand_data['month'] != 12)
]
voronoi_grid_demand_data.head()

voronoi_grid_id,startTime,0,1,2,3,4,5,6,7,8,...,1142,1144,1145,1146,1147,1148,1149,hour,day_of_week,month
0,2017-10-14 12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,5,10
1,2017-10-14 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,15,5,10
2,2017-10-14 18:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,5,10
3,2017-10-14 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,5,10
4,2017-10-15 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,6,10


In [18]:
voronoi_grid_community_dict = input_training_data.groupby('community_voronoi_grid_id')['voronoi_grid_id'].apply(lambda x: list(set(x))).to_dict()

In [19]:
voronoi_grid_poisson_predicted_demand = pd.DataFrame()
voronoi_grid_negative_binomial_predicted_demand = pd.DataFrame()

voronoi_grid_poisson_predicted_demand['timestamp'] = voronoi_grid_demand_data.startTime
voronoi_grid_negative_binomial_predicted_demand['timestamp'] = voronoi_grid_demand_data.startTime

for (community, cells) in voronoi_grid_community_dict.items():
    print(f'Training model for community {community}')

    if voronoi_grid_models.poisson_model[community] is None:
        print(f'Error model is none skipping community {community}')
        continue

    x = small_grid_demand_data[['hour', 'day_of_week', 'month']]
    x = pd.get_dummies(x, columns=['hour', 'day_of_week', 'month'], drop_first=True)


    exog = x.copy()
    columns = voronoi_grid_models.poisson_model[community].model.exog_names

    # Collect all columns to be added in a list
    columns_to_add = []

    for col in columns:
        if col not in exog.columns:
            columns_to_add.append(pd.Series(False, name=col, index=exog.index))

    # Concatenate all columns at once
    if columns_to_add:
        exog = pd.concat([exog] + columns_to_add, axis=1)

    for cell in cells:
        if 'grid_cell_' + str(cell) in exog.columns:
            exog['grid_cell_' + str(cell)] = True
        poisson_prediction = voronoi_grid_models.poisson_model[community].predict(exog)
        negative_binomial_prediction = voronoi_grid_models.negative_binomial_model[community].predict(exog)

        voronoi_grid_poisson_predicted_demand[cell] = poisson_prediction
        voronoi_grid_negative_binomial_predicted_demand[cell] = negative_binomial_prediction

voronoi_grid_poisson_predicted_demand.head()

Training model for community 0
Training model for community 1
Training model for community 2
Training model for community 3
Training model for community 4
Training model for community 5
Training model for community 6
Training model for community 7
Training model for community 8
Training model for community 9
Training model for community 10
Training model for community 11
Training model for community 12
Training model for community 13
Training model for community 14
Error model is none skipping community 14
Training model for community 15
Training model for community 16
Training model for community 17
Training model for community 18
Training model for community 19


,timestamp,1025,515,6,524,525,526,1038,529,1041,...,1142,893,808,1032,1035,815,1077,954,1054,996
0,2017-10-14 12:00:00,0.005088,0.000025,0.000025,1.195829e-06,3.441120e-08,9.325959e-10,5.810045e-13,4.433931e-14,8.017545e-16,...,7.508271e-34,3.201612e-35,3.263668e-05,8.035639e-06,3.554245e-07,6.250509e-08,5.579297e-09,5.025693e-10,4.525012e-11,7.044250e-10
1,2017-10-14 15:00:00,0.006600,0.000033,0.000033,1.551103e-06,4.463456e-08,1.209665e-09,7.536175e-13,5.751225e-14,1.039951e-15,...,1.108006e-33,4.724662e-35,8.024390e-05,1.975725e-05,8.738833e-07,1.536815e-07,1.371784e-08,1.235669e-09,1.112566e-10,6.631421e-10
2,2017-10-14 18:00:00,0.003637,0.000018,0.000018,8.549252e-07,2.460135e-08,6.667340e-10,4.153733e-13,3.169917e-14,5.731925e-16,...,8.527597e-34,3.636264e-35,8.804158e-07,2.167715e-07,9.588026e-09,1.686154e-09,1.505086e-10,1.355744e-11,1.220679e-12,6.269652e-08
3,2017-10-14 21:00:00,0.001287,0.000006,0.000006,3.024057e-07,8.702033e-09,2.358383e-10,1.469266e-13,1.121269e-14,2.027507e-16,...,2.007545e-34,8.560399e-36,8.589199e-07,2.114789e-07,9.353928e-09,1.644985e-09,1.468339e-10,1.322643e-11,1.190876e-12,9.501143e-10
4,2017-10-15 06:00:00,0.001280,0.000006,0.000006,3.008456e-07,8.657141e-09,2.346217e-10,1.461686e-13,1.115485e-14,2.017047e-16,...,9.135225e-35,3.895364e-36,1.700218e-04,4.186192e-05,1.851595e-06,3.256223e-07,2.906553e-08,2.618151e-09,2.357320e-10,7.387559e-10


In [20]:
voronoi_grid_negative_binomial_predicted_demand.head()

,timestamp,1025,515,6,524,525,526,1038,529,1041,...,1142,893,808,1032,1035,815,1077,954,1054,996
0,2017-10-14 12:00:00,0.005919,0.000017,0.000017,5.910973e-07,1.322637e-08,2.855167e-10,1.414102e-13,8.077077e-15,1.245703e-16,...,8.073405e-34,3.220975e-35,0.000039,9.306159e-06,4.165125e-07,7.740528e-08,7.094335e-09,6.336122e-10,5.713783e-11,7.967691e-08
1,2017-10-14 15:00:00,0.007769,0.000022,0.000022,7.759043e-07,1.736160e-08,3.747836e-10,1.856222e-13,1.060238e-14,1.635173e-16,...,1.234648e-33,4.925766e-35,0.000082,1.983830e-05,8.878958e-07,1.650078e-07,1.512327e-08,1.350696e-09,1.218029e-10,7.750515e-08
2,2017-10-14 18:00:00,0.004225,0.000012,0.000012,4.219116e-07,9.440677e-09,2.037952e-10,1.009353e-13,5.765231e-15,8.891544e-17,...,9.320630e-34,3.718569e-35,0.000001,2.854837e-07,1.277729e-08,2.374550e-09,2.176319e-10,1.943723e-11,1.752809e-12,8.458035e-07
3,2017-10-14 21:00:00,0.001478,0.000004,0.000004,1.476155e-07,3.303037e-09,7.130243e-11,3.531454e-14,2.017098e-15,3.110911e-17,...,2.201057e-34,8.781364e-36,0.000001,2.845105e-07,1.273374e-08,2.366456e-09,2.168900e-10,1.937097e-11,1.746834e-12,9.187607e-08
4,2017-10-15 06:00:00,0.001467,0.000004,0.000004,1.465274e-07,3.278691e-09,7.077686e-11,3.505424e-14,2.002230e-15,3.087980e-17,...,9.946897e-35,3.968425e-36,0.000191,4.602148e-05,2.059767e-06,3.827901e-07,3.508341e-08,3.133384e-09,2.825621e-10,7.953740e-08


In [21]:
voronoi_grid_poisson_predicted_demand.to_pickle(voronoi_grid_poisson_demand_output_filepath)
voronoi_grid_negative_binomial_predicted_demand.to_pickle(voronoi_grid_negative_binomial_demand_output_filepath)